# Capstone Problem statement 2 using SMOTE and sklearn Python libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

review = pd.read_csv('/content/reviews.csv')
review

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,15416,V Jillian,Holly's place was great. It was exactly what I...
...,...,...,...,...,...,...
392954,1219622445222452442,1230654319092135704,2024-08-24,51979490,Solène,D
392955,1219622445222452442,1232036398314385400,2024-08-26,18919150,Kristopher,Great stay. Key vob stopped working but host a...
392956,1219785829994931401,1237167935539710683,2024-09-02,4338272,Anastasia,Great choice for staying! The apartment is ama...
392957,1233574822709070023,1237138800668239871,2024-09-02,541938241,Dave,Wonderful


In [2]:
review.shape

(392959, 6)

In [5]:
review.value_counts()

listing_id           id                   date        reviewer_id  reviewer_name  comments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
958                  5977                 2009-07-23  15695        Edmund C       Our experience was, without a doubt, a five star experience. Holly and her husband, David, were the consummate hosts; friendly and accomodating while still honoring our privacy. The apartment was a charming layout with a full view and access to the home's garden The location is perfect for full engagement with the city; close to mass transit with walking proximity to the Haight, the Mission, the Castro and Golden Gate Park. I can't wait for our next visit.\r<br/>\r<br/>Ted and Karen Wingerd                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         1
30069424             624869237            2020-05-15  81954409     Janiqua        I’ll start with the good. This is a really great place as far as getting an apt in a nice area, with a garage parking spot that you can actually fit a car in! It is also full of all the amenities you could need. The bathrooms are GORGEOUS! You also share the W/D with the neighbor, but still very convenient. Edwin is very responsive and always answered in a timely manner. On the flip side, there were more spiders than I cared to see. The walls were pretty scuffed when we arrived as well in nearly every room, so we had to magic erase those ourselves unfortunately. But the worst part of this stay was the upstairs neighbor. Beware! He sings nearly everyday, at the top of his lungs, with complete disregard for the people below him. It wouldn’t be so bad if you couldn’t hear him so clearly, but unfortunately you can and that’s even with ear plugs in. In addition, it’s just a lot of other miscellaneous noise that became the norm and got old quick.  Keep in mind I stayed here for 3 mos, so if you’re in and out you might not experience all of these issues. Just wanted to give a heads up I wish I had.    1
30091565             353128547            2018-11-26  12429185     Daylon         Good spot for a quick weekend stay in a central part of the city!                                                                                                                                                                                                                                                                              

#Adding sentiment score and proceed with imbalanced dataset

In [7]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392959 entries, 0 to 392958
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     392959 non-null  int64 
 1   id             392959 non-null  int64 
 2   date           392959 non-null  object
 3   reviewer_id    392959 non-null  int64 
 4   reviewer_name  392956 non-null  object
 5   comments       392889 non-null  object
dtypes: int64(3), object(3)
memory usage: 18.0+ MB


In [8]:
review.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [9]:
review.describe()

,listing_id,id,reviewer_id
count,3.929590e+05,3.929590e+05,3.929590e+05
mean,8.225927e+16,4.223165e+17,1.251442e+08
std,2.459846e+17,4.557533e+17,1.407015e+08
min,9.580000e+02,1.981000e+03,1.000000e+00
25%,4.359078e+06,2.791330e+08,1.992458e+07
50%,1.742188e+07,7.701426e+08,6.663371e+07
75%,3.680482e+07,8.646046e+17,1.818866e+08
max,1.234137e+18,1.237913e+18,5.984690e+08


# Importing NLTK for sentiment score analysis.

In [10]:
pip install nltk

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Load the sentiment analyzer
sent = SentimentIntensityAnalyzer()
review_final = pd.read_csv('/content/reviews.csv', usecols=['comments'])
review_final['comments'] = review_final['comments'].astype(str)

# Calculate polarity scores
polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in review_final['comments']]
review_final['sentiment_score'] = polarity
review_final.head()

,comments,sentiment_score
0,"Our experience was, without a doubt, a five st...",0.96
1,Returning to San Francisco is a rejuvenating t...,0.98
2,We were very pleased with the accommodations a...,0.76
3,We highly recommend this accomodation and agre...,0.98
4,Holly's place was great. It was exactly what I...,0.96


In [13]:
review_final.shape

(392959, 2)

In [14]:
review_final.sentiment_score.value_counts()

,count
sentiment_score,
0.98,33199
0.97,32099
0.96,27620
0.99,25300
0.95,22797
...,...
-0.11,25
0.07,20
-0.01,19


In [16]:
x = review.iloc[:,:-1]
y = review_final.sentiment_score
x.head()

,listing_id,id,date,reviewer_id,reviewer_name
0,958,5977,2009-07-23,15695,Edmund C
1,958,6660,2009-08-03,26145,Simon
2,958,11519,2009-09-27,25839,Denis
3,958,16282,2009-11-05,33750,Anna
4,958,26008,2010-02-13,15416,V Jillian


# Label encoding we are doing for the variables with string values

In [32]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
x.loc[:,['reviewer_name', 'date']] = x.loc[:,['reviewer_name', 'date']].apply(enc.fit_transform)
y=enc.fit_transform(y)
x.head(100)

,listing_id,id,date,reviewer_id,reviewer_name
0,958,5977,4,15695,11106
1,958,6660,5,26145,39220
2,958,11519,15,25839,9810
3,958,16282,19,33750,2635
4,958,26008,31,15416,43411
...,...,...,...,...,...
95,958,172021239,2513,5881546,35598
96,958,174401034,2519,122131082,8757
97,958,177060286,2526,16752321,28257
98,958,178027423,2529,55728647,22007


# Previous step has concerns. Need to be discussed for sure.

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 10)

In [34]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors= 10)
knn.fit(x_train,y_train)
y_predict = knn.predict(x_test)

In [36]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.05150651465798046


col_0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
row_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,2,2,0,1,1,4,2,7,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,1,0,0,2,0,3,6,5,4,3,3,2,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,3,2,1,6,10,10,9,1,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,4,3,3,2,5,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,1,6,5,3,11,1,3,7,14,0,3,0,1,0,5,4,11,4,1,0,4,2,1,2,0,0,2,2,0,2,2,0,4,2,1,8,1,1,3,21,5,0,1,10,1,2,4,2,0,0,...,14,3,3,5,0,5,15,1,6,13,8,77,1,8,4,63,7,7,6,2,6,7,12,14,3,13,20,37,17,34,36,21,39,59,54,55,80,69,118,130,137,264,258,335,390,491,652,613,317,8
197,5,9,5,2,15,4,2,2,9,2,2,2,2,2,2,2,15,1,4,2,0,5,0,1,1,1,4,0,2,6,4,2,5,2,2,7,2,1,1,11,0,1,1,5,1,1,6,1,3,1,...,10,4,7,9,4,7,23,1,12,10,11,115,0,7,6,75,5,7,12,3,8,8,6,12,8,13,20,49,25,40,36,32,41,61,54,66,90,76,139,167,159,258,280,406,434,554,722,682,370,4
198,1,6,9,2,11,4,5,7,12,1,1,3,2,1,1,2,18,1,7,3,1,3,2,1,2,1,1,4,1,5,2,3,2,3,2,5,0,3,0,17,1,0,3,5,1,5,7,1,2,1,...,13,5,0,6,4,7,26,4,14,12,14,117,0,15,7,57,9,8,10,5,10,12,13,17,10,14,22,33,21,37,46,22,47,63,69,64,86,81,103,153,163,288,343,409,488,645,731,717,435,8
